In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  3 20:01:22 2020

@author: mor
"""

import re
import nltk
import codecs
import string
import subprocess
import unicodedata
import os
from io import StringIO
import pandas as pd
from operator import itemgetter
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob



In [ ]:
plt.style.use('ggplot')


##  Data preprocessing , docx format





In [ ]:

## Set important directory
BASE = os.getcwd()
DOCS = os.path.join(BASE, "data")
## Create a path to extract the corpus.
CORPUS = os.path.join(BASE, "data", "corpus")
MODELS = os.path.join(BASE,'models')



### Returns a filtered list of paths to .docx files representing our corpus.


In [ ]:
def get_documentsDOC(path=DOCS):
    for name in os.listdir(path):
        if name.endswith('.docx'):
            yield os.path.join(path, name)


def get_documentsDOC2(path=DOCS):
    for file_path in glob.glob(DOCS+'\\*.docx'):
        yield file_path

In [ ]:
         
# Print the total number of documents
print(len(list(get_documentsDOC())))
   

In [ ]:

path='..\\data\\8b3f32dc3f214f1f9ba0bfa717703900.docx'
path2='..\\data\\1a016400391c4cd4a1926407ad02fe1a.docx'



Module that extract text from MS XML Word document (.docx).
(Inspired by python-docx <https://github.com/mikemaccana/python-docx>)


In [ ]:
try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile
from io import BytesIO, StringIO               
from docx import Document   


WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'



#Take the path of a docx file, return the body text
def get_docx_text(path):

    document = zipfile.ZipFile(path)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.getiterator(PARA):
        texts = [node.text.encode('utf-8', 'ignore').decode('utf-8')
                 for node in paragraph.getiterator(TEXT)
                 if node.text]
        if texts:
            texts=standardize_text(texts)
            paragraphs.append(''.join(texts))
            paragraphs=standardize_text(paragraphs)
    return '\n'.join(paragraphs)

#Take the path of a docx file, return the header text
def get_headers(path):
    document = zipfile.ZipFile(path)
    headers = []
    for name in document.namelist():
        if 'header' in name:
            xml_content = document.read(name)    
            tree = XML(xml_content)

            
            for paragraph in tree.getiterator(PARA):
                texts = [node.text.encode('utf-8', 'ignore').decode('utf-8')
                         for node in paragraph.getiterator(TEXT)
                         if node.text]
                if texts:
                    texts=standardize_text(texts)
                    headers.append(' '.join(texts))
                    headers=standardize_text(headers)
    document.close()
    return '\n'.join(headers)


def standardize_text(texts):
    newTexts=[]
    for t in texts:
#        t=t.strip('\s')
        t = re.sub(r"[^A-Za-z0-9(),!?@\':\`\-\.\"\_\na-z\u0590-\u05fe]", ' ', t)
        t = t.lower()
        t=re.sub(r"[0-9]+[\-:\s][0-9]+([\-:\s][0-9]+)?\s","",t)
        t=  t.encode('utf-8', 'ignore').decode('utf-8')
        t=  re.sub('\s+',' ',t)
        newTexts.append(t)
    return newTexts

In [ ]:

print('header')
print(get_headers(path2))
print('\ntext')
print(get_docx_text(path2))




### Create txt file for every docx file



In [ ]:

#Extracts a text corpus from the docx documents and writes them to disk as a .txt.
def extract_corpus(docs=DOCS, corpus=CORPUS):
#    i=0
    # Create corpus directory if it doesn't exist.
    if not os.path.exists(corpus):
        os.mkdir(corpus)
    
    for path in get_documentsDOC(docs):
        print(path)
        document = get_headers(path)
        document = document + (get_docx_text(path))
        document=document.replace('\n',' ')
        document=standardize_text([document])[0]
        
        fname = os.path.splitext(os.path.basename(path))[0] + ".txt"
        outpath = os.path.join(corpus, fname)
        with codecs.open(outpath, 'w', encoding='utf-8', errors='replace') as f: #, encoding='utf-8', errors='replace'
            f.write(document)
    print('******************** preprocess docx files   -  end *******************')
    print('******************** created txt files -  end *******************')
    


In [ ]:
          
# Run the extraction of text
extract_corpus()






###  Data preprocessing txt file





In [ ]:

def readFiles(path):
    data=[]
    filesNames=[]
    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            with codecs.open(os.path.join(path,filename), 'r',encoding="utf-8") as content_file:
                file_content = content_file.read()
    #            print(str(file_handle.name))
    #            print(file_content)
                filesNames.append(str(filename))
                data.append(file_content)
    
    return filesNames,data


In [ ]:
    
filesnames,articles=readFiles(CORPUS)


In [ ]:
# =============================================================================
# Files to pandas dataframe
# =============================================================================

basic = pd.DataFrame(list(zip(filesnames,articles)),columns=['filename','text'])


In [ ]:

def standardize_textDF(df, text_field):
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z,!?@\'\.\"\_\na-z\u0590-\u05fe]", " ")
    df[text_field] = df[text_field].str.replace(r'\s+'," ")
    df[text_field] = df[text_field].str.strip()
    return df[text_field]


In [ ]:

basic['textclean']=standardize_textDF(basic,'text')


In [ ]:

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\S+')

basic["tokens"] = basic["text"].apply(tokenizer.tokenize)
basic["tokensclean"] = basic["textclean"].apply(tokenizer.tokenize)


In [ ]:
 
def removeStartAndEndPunctuation (tokens_list):
    tokens_list = [re.sub(r"^[^A-Za-za-z\u0590-\u05fe]+", "",word) for word in tokens_list]
    tokens_list = [re.sub(r"[^A-Za-za-z\u0590-\u05fe]+$", "",word) for word in tokens_list]
    tokens_list = [word for word in tokens_list if word != '']
    tokens_list = [word for word in tokens_list if len(word) > 1]
    return tokens_list


In [ ]:

basic['tokenNoPunctuation']=basic['tokensclean'].apply(lambda x: removeStartAndEndPunctuation(x))
